# Multilingual Named Entity Recognition

In [82]:
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset
import numpy as np
import torch


from transformers import AutoConfig
from transformers import AutoTokenizer
from transformers import DataCollatorForTokenClassification
from transformers import Trainer
from transformers import TrainingArguments

from seqeval.metrics import f1_score


import torch.nn as nn
from transformers import BertConfig
from transformers.modeling_outputs import TokenClassifierOutput
from transformers.models.bert.modeling_bert import BertModel
from transformers.models.bert.modeling_bert import BertPreTrainedModel

## Loading the Dataset

In [83]:
train = pd.read_parquet("data/train-00000-of-00001.parquet")
dev = pd.read_parquet("data/dev-00000-of-00001.parquet")
test = pd.read_parquet("data/test-00000-of-00001.parquet")


### convert to Huggingface dataset
train_dataset = Dataset(pa.Table.from_pandas(train))
dev_dataset = Dataset(pa.Table.from_pandas(dev))
test_dataset = Dataset(pa.Table.from_pandas(test))

In [84]:
tags = ["O"] + list(set([x[0]["label"] for x in train_dataset["ents"] if x]))
index2tag = {idx: tag for idx, tag in enumerate(tags)}
tag2index = {tag: idx for idx, tag in enumerate(tags)}

## Multilingual Transformers

In [85]:
bert_model_name = "bert-base-multilingual-cased"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)

### Model Definition

In [86]:
class BertForTokenClassification(BertPreTrainedModel):
    config_class = BertConfig

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = len(tags)
        # Load model body
        self.bert = BertModel(config, add_pooling_layer=False)
        # Set up token classification head
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, self.num_labels)
        # Load and initialize weights
        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        labels=None,
        **kwargs
    ):
        # Use model body to get encoder representations
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            **kwargs
        )
        # Apply classifier to encoder representation
        sequence_output = self.dropout(outputs[0])
        logits = self.classifier(sequence_output)
        # Calculate losses
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
        # Return model output object
        return TokenClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

### Loading pretrained Bert

In [87]:
bert_config = AutoConfig.from_pretrained(
    bert_model_name, num_labels=len(tags), id2label=index2tag, label2id=tag2index
)

In [88]:
# device = torch.device("cpu")
device = torch.device("mps")

bert_model = BertForTokenClassification.from_pretrained(
    bert_model_name, config=bert_config
).to(device)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [89]:
def tag_text(text, tags, model, tokenizer):
    # Get tokens with special characters
    tokens = tokenizer(text).tokens()
    # Encode the sequence into IDs
    input_ids = bert_tokenizer(text, return_tensors="pt").input_ids.to(device)
    # Get predictions as distribution over 7 possible classes
    outputs = model(input_ids)[0]
    # Take argmax to get most likely class per token
    predictions = torch.argmax(outputs, dim=2)
    # Convert to DataFrame
    preds = [tags[p] for p in predictions[0].cpu().numpy()]
    return pd.DataFrame([tokens, preds], index=["Tokens", "Tags"])

## Tokenizing Texts for NER

In [90]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = bert_tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    labels = []
    for idx, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=idx)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None or word_idx == previous_word_idx:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

## Encode Dataset as tokens

In [91]:
def encode_line(line):
    ents = line["ents"]
    tokenized = bert_tokenizer(line["text"])
    labels = ["O"]

    word_start = 0
    for word in bert_tokenizer.convert_ids_to_tokens(tokenized["input_ids"]):
        if word in ("[CLS]", "[SEP]"):
            continue
        if word.startswith("##"):
            word = word[2:]

        word_start += line["text"][word_start:].find(word)

        if ents:
            if word_start >= ents[0]["start"] and word_start <= ents[0]["end"]:
                labels.append(ents[0]["label"])
            else:
                labels.append("O")
            if ents[0]["end"] <= word_start + len(word):
                ents = ents[1:]
        else:
            labels.append("O")

        # print(line["text"][word_start: word_start+len(word)])
        word_start += len(word)

    labels = [tag2index[x] for x in labels + ["O"]]
    tokenized["labels"] = labels
    # print(pd.DataFrame([i for i in zip(bert_tokenizer.convert_ids_to_tokens(tokenized["input_ids"]),labels)]))
    return tokenized


train_dataset = pd.DataFrame([encode_line(l) for l in train_dataset])
dev_dataset = pd.DataFrame([encode_line(l) for l in dev_dataset])
test_dataset = pd.DataFrame([encode_line(l) for l in test_dataset])

# train_dataset = Dataset(pa.Table.from_pandas(train_dataset))
# dev_dataset = Dataset(pa.Table.from_pandas(dev_dataset))
# test_dataset = Dataset(pa.Table.from_pandas(test_dataset))

In [92]:
train_dataset

,attention_mask,input_ids,labels,token_type_ids
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 17271, 17183, 78763, 10107, 10452, 10742...","[0, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 145, 21146, 177, 24069, 47378, 118, 1015...","[0, 0, 0, 0, 0, 0, 0, 2, 0, 14, 14, 14, 14, 14...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 150, 76525, 10112, 10349, 17731, 11400, ...","[0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 31826, 10453, 11443, 15935, 108656, 1010...","[0, 0, 0, 0, 12, 12, 12, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[1, 1, 1, 1, 1, 1, 1]","[101, 196, 11459, 16081, 24188, 38308, 102]","[0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...
11757,"[1, 1, 1, 1, 1, 1]","[101, 21574, 17640, 12730, 119, 102]","[0, 8, 8, 8, 0, 0]","[0, 0, 0, 0, 0, 0]"
11758,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[101, 14321, 11478, 49210, 83343, 18797, 10163...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
11759,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 11459, 53265, 47124, 10136, 24774, 10570...","[0, 0, 0, 11, 11, 1, 1, 1, 5, 5, 5, 5, 5, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11760,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[101, 30186, 74589, 46111, 10419, 15838, 78987...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [93]:
def get_torch_dataset(df):
    data_set = []

    print(df)
    for row in df.iloc:
        data_set.append({key: torch.tensor(row[key]).to(device) for key in row.keys()})
    return data_set

# training_set = get_torch_dataset(training_df)
# dev_set = get_torch_dataset(dev_df)

train_dataset = get_torch_dataset(train_dataset)
dev_dataset = get_torch_dataset(dev_dataset)
test_dataset = get_torch_dataset(test_dataset)

                                          attention_mask  \
0      [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
1      [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
2      [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
3      [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
4                                  [1, 1, 1, 1, 1, 1, 1]   
...                                                  ...   
11757                                 [1, 1, 1, 1, 1, 1]   
11758         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]   
11759  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
11760               [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]   
11761  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   

                                               input_ids  \
0      [101, 17271, 17183, 78763, 10107, 10452, 10742...   
1      [101, 145, 21146, 177, 24069, 47378, 118, 1015...   
2      [101, 150, 76525, 10112, 10349, 17731, 11400, ...   
3      [101, 31826, 10453, 11443, 15935

## Performance Measures

In [94]:
def align_predictions(predictions, label_ids):
    preds = np.argmax(predictions, axis=2)
    batch_size, seq_len = preds.shape
    labels_list, preds_list = [], []

    for batch_idx in range(batch_size):
        example_labels, example_preds = [], []
        for seq_idx in range(seq_len):
            # Ignore label IDs = -100
            if label_ids[batch_idx, seq_idx] != -100:
                example_labels.append(index2tag[label_ids[batch_idx][seq_idx]])
                example_preds.append(index2tag[preds[batch_idx][seq_idx]])

        labels_list.append(example_labels)
        preds_list.append(example_preds)

    return preds_list, labels_list

## Training

In [95]:
# num_epochs = 1
# batch_size = 30
# logging_steps = len(train_dataset) // batch_size
# model_name = f"{bert_model_name}-finetuned-panx-de"
# training_args = TrainingArguments(
#     output_dir=model_name,
#     log_level="error",
#     num_train_epochs=num_epochs,
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     evaluation_strategy="epoch",
#     save_steps=1e6,
#     weight_decay=0.01,
#     disable_tqdm=False,
#     logging_steps=logging_steps,
#     push_to_hub=False,
# )

In [96]:
train_params = {'batch_size': 30,
                'shuffle': True,
                'num_workers': 0,
                # 'multiprocessing_context':'fork' 
                }


training_loader = torch.utils.data.DataLoader(train_dataset, **train_params)

In [97]:
def compute_metrics(eval_pred):
    y_pred, y_true = align_predictions(eval_pred.predictions, eval_pred.label_ids)
    return {"f1": f1_score(y_true, y_pred)}

In [98]:
data_collator = DataCollatorForTokenClassification(bert_tokenizer)

In [99]:
def model_init():
    return BertForTokenClassification.from_pretrained(
        bert_model_name, config=bert_config
    ).to(device)

In [100]:
%env TOKENIZERS_PARALLELISM=false

env: TOKENIZERS_PARALLELISM=false


In [101]:
# trainer = Trainer(
#     model_init=model_init,
#     args=training_args,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,
#     train_dataset=train_dataset,
#     eval_dataset=dev_dataset,
#     tokenizer=bert_tokenizer,
# )

In [102]:
# trainer.is_model_parallel
model = model_init()

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [103]:
from tqdm import tqdm
N_EPOCHS = 1



def run_epoch(epoch):
    model.train()

    tr_loss, tr_accuracy = 0, 0

    for idx, batch in enumerate(tqdm(training_loader)):
        ids = batch["input_ids"].to(device, dtype=torch.long)
        mask = batch["attention_mask"].to(device, dtype=torch.long)
        targets = batch["labels"].to(device, dtype=torch.long)
        
        outputs = model(input_ids = ids,
                        attention_mask = mask,
                        labels = targets)
        
        loss, tr_logits = outputs.loss, outputs.logits
        tr_loss += loss.item()

        flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
        active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
        targets = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


for epoch in range(N_EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    run_epoch(epoch)

Training epoch: 1


  0%|          | 0/393 [00:00<?, ?it/s]


RuntimeError: stack expects each tensor to be equal size, but got [11] at entry 0 and [16] at entry 1

In [105]:
for idx, batch in tqdm(training_loader):
    print(batch)

  0%|          | 0/393 [00:00<?, ?it/s]


RuntimeError: stack expects each tensor to be equal size, but got [11] at entry 0 and [27] at entry 1

In [ ]:
# hide_output
text_de = (
    "Jeff Dean ist ein Informatiker bei Google in Kalifornien, syntes jeg næste år"
)
tag_text(text_de, tags, trainer.model, bert_tokenizer)

RuntimeError: Placeholder storage has not been allocated on MPS device!